In [57]:
import sys
sys.path.append('/Users/Jessica/Desktop/Coding/School/2802ICT/Assignment2')

from imports import *
from collections import Counter

In [58]:
class Node:     #this class creats an object for each node in the tree

    def __init__(self, feature=None, thresh=None, left=None, right=None, *, value=None):        
        self.feature = feature
        self.value = value
        self.right_child = right
        self.left_child = left
        self.thresh = thresh
        self.labels = []

    def isNode(self):
        if self.value is not None:
            return True
        else:
            return False

In [59]:

class DecisionTree: 

    def __init__(self, data, max_depth, min_split = 2, num_feats = None):
        self. df = data

        self.feature_train = []
        self.feature_test = []
        self.target_train = []
        self.target_test = []

        self.max_depth = max_depth
        self.min_split = min_split
        #given dataset has 6 features but can use datasets
        self.num_feats = num_feats 
        self.root = None

    def accuracy(self, split):

        self.split_data(split)
        self.fit()
        
        
    def fit(self):
        self.num_feats = self.feature_train.shape[1]
        self.root = self.build_tree(self.feature_train, self.target_train)

    #builds the tree recursively
    def build_tree(self, f_train, t_train, depth = 0):
        
        #amount of samples in dataset
        num_samples = f_train.shape[0]
        #amount of features
        num_feats = self.num_feats
        #num of labels -> (acc/unacc/good/vgood)
        n_labels = len(np.unique(t_train))
        #criteria that end the searching
        if (depth >= self.max_depth or n_labels == 1 or num_samples < self.min_split):
            leaf = self.high_freq(t_train)             
            return Node(value = leaf)
        
        feature_ids = np.random.choice(num_feats, self.num_feats, replace=False)
        
        if f_train.size != 0 and t_train.size != 0:
            # finds the best feature to split on and the threshold
            best_feature, threshold = self.best_split(f_train, t_train, feature_ids)
            
            #where to split to children nodes
            #left_id, right_id = self.child_split(f_train[:, best_feature], threshold)

    def best_split(self, f_train, t_train, f_ids):
        
        best_gain = -1
        split_id, split_threshold = None, None
        
        for f_id in f_ids:
            #f_id_col holds the column for 1 feature at a time
            f_id_col = f_train[:, f_id]

            #thresholds has the unique elements in each feature column
            thresholds = np.unique(f_id_col)

            for threshold in thresholds:
                gain = self.info_gain(t_train, f_id_col, threshold)

                if gain > best_gain:

                    best_gain = gain
                    split_id = f_id
                    split_threshold = threshold

        return split_id, split_threshold


    def info_gain(self, t_train, feature_col, threshold):
        parent_entropy = self.entropy(t_train)
        print(parent_entropy)
        left_id, right_id = self.next_split(feature_col, threshold)

    
    #finds the entropy -> fequency/length 
    def entropy(self, t_train):
        histogram = []
        counted_vals = Counter(t_train)
        for word in set(self.target_train):
            new_val = counted_vals[word]
            histogram.append(new_val)

        histogram = np.array(histogram)
        length = len(t_train)              
        calc = histogram/length
        
        return -(np.sum([c * np.log2(c) for c in calc if c > 0]))


    def split_data(self, split):
        dataset = np.array(self.df)
        #randomly shuffle dataset
        np.random.shuffle(dataset)  

        #multiply length by wanted percent and // 1 to ensure its a wh0le number
        test_size = int((len(dataset)*split)//1)                      
        train_size = int(len(dataset) - test_size)

        #use the length to make respective lists
        train = dataset[:train_size]                                     
        test = dataset[test_size:]

        #only want the last column. THEN map -> join as cars are strings
        target_train = np.delete(train, np.s_[0:6], axis = 1)                  
        self.target_train = np.array(list(map(''.join, target_train)))

        self.feature_train = np.delete(train, 6, axis = 1)                  
        #self.feature_train = np.array(list(map(''.join, feature_train)))

        self.target_test = np.delete(test, np.s_[0:6], axis = 1)
        #self.target_test = np.array(list(map(''.join, target_test)))

        self.feature_test = np.delete(test, 6, axis = 1)
        #self.feature_test = np.array(list(map(''.join, feature_test)))

    def high_freq(self, target_train):   
        counter = Counter(target_train)  
        #most common value present which is useful when determing where to split  
        most_common = counter.most_common(1)[0][0] 
        return most_common

In [60]:

if __name__ == "__main__":

    #file is read from a class called SetUp
    data = SetUp("car.csv", {'buying':'string','maint':'string','doors':'string','persons':'string','lug_boot':'string','safety':'string','class':'string'}) 

    ####################### USER SETUP AREA #######################
    
    split = 0.25
    max_depth = 5
    print_each_k_percentage = False             #if you want to print each k's percentage    
    
    ####################### END OF USER SETUP #######################

    DT = DecisionTree(data.df, max_depth)
    DT.accuracy(split)





1.2117705715535723


AttributeError: 'DecisionTree' object has no attribute 'next_split'